The training loop code for this model was adapted from https://colab.research.google.com/drive/13dZVYEOMhXhkXWfvSMVM1TTtUDrT6Aeh, which sets up a basic scheme for loading data and finetuning a model. All other preprocessing and evaluation code is my own. I have marked the sections taken from their demo.





# Setup

In [ ]:
!pip install transformers
!pip install evaluate

In [ ]:
import os
import time
import datetime
from google.colab import drive
from tqdm import tqdm

import pandas as pd
import numpy as np
import random

import torch
import spacy
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

import nltk
nltk.download('punkt')
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/marcoguerini/CONAN/master/Multitarget-CONAN/Multitarget-CONAN.csv')

In [ ]:
nlp = spacy.load("en_core_web_sm")

entities = []
for i in tqdm(range(5003)):
    doc = nlp(df["HATE_SPEECH"][i])
    l = ""
    if len(doc.ents) > 0:
        for ent in doc.ents:
            l += "_".join(ent.text.split(" ")) + "_" + ent.label_ + " "
    entities.append(l)

df = df.assign(textentities = entities)

In [ ]:
import pickle
with open('rationales.pkl', 'rb') as f:
    rats = pickle.load(f)

In [ ]:
df.dropna(inplace=True) #remove NA values
df["combined"] = df["HATE_SPEECH"] + "<|eohs|>" + df["textentities"] + "<|eone|>" + df["COUNTER_NARRATIVE"]
data = df["combined"]
data2 = df["HATE_SPEECH"]
data3 = df["COUNTER_NARRATIVE"]

# GPT2 Tokenizer

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium

In [ ]:
batch_size = 2

In [ ]:
# provided by reference code

class GPT2Dataset(Dataset):

  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=768):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list:

      encodings_dict = tokenizer('<|startoftext|>'+ txt + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx] 

In [ ]:
dataset = GPT2Dataset(data, tokenizer, max_length=768)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

In [ ]:
train_dataset, val_dataset, t1, v1, t2, v2 = train_test_split(dataset, data2, data3, test_size=0.1)

In [ ]:
train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = batch_size)

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

# Finetune GPT2 Language Model; the following code is taken from the reference code but has been adapted to fit the specific task

In [ ]:
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

model.resize_token_embeddings(len(tokenizer))

device = torch.device("cuda")
model.cuda()

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

sample_every = 100

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

In [ ]:
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

In [ ]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            
            eohs = "<|eohs|>"
            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0
    scores = []

    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device) #
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        
        with torch.no_grad():        

            outputs = model(b_input_ids, 
                             attention_mask = b_masks,
                            labels=b_labels)
          
            loss = outputs[0]  
            
        
        batch_loss = loss.item()
        total_eval_loss += batch_loss  
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

# Saving & Loading Fine-Tuned Model


In [ ]:
output_dir = './model_save/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

model_to_save = model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!cp -r ./model_save/ drive/MyDrive/models

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

In [ ]:
# Copy the model files to a directory in your Google Drive.
#!cp -r ./model_save/ gdrive/MyDrive/models

# # Load a trained model and vocabulary that you have fine-tuned
model = GPT2LMHeadModel.from_pretrained('.')
tokenizer = GPT2Tokenizer.from_pretrained('.')
model.to(device)

In [ ]:
from tqdm import tqdm

model.eval()

outputs = []
for seq in tqdm(v1):
  prompt =  "<|startoftext|>" + seq + "<|eohs|>"

  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  sample_outputs = model.generate(
                                  generated, 
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length = 300,
                                  top_p=0.95, 
                                  num_return_sequences=1, pad_token_id=tokenizer.eos_token_id
                                  )

  for i, sample_output in enumerate(sample_outputs):
    outputs.append(tokenizer.decode(sample_output, skip_special_tokens=True).split("<|eohs|>")[1])

In [ ]:
from nltk.translate import bleu_score
print(bleu_score.corpus_bleu(v2.tolist(),
    outputs,
    weights=(1, 0, 0, 0),
    smoothing_function=None,
    auto_reweigh=False,
))

In [ ]:
!pip install rouge_score
rouge = evaluate.load('rouge')
r = rouge.compute(predictions=outputs, references=v2.tolist())
!pip install meteor_score
meteor = evaluate.load('meteor')
m = meteor.compute(predictions=outputs, references=v2.tolist())

In [ ]:
!cp test.zip gdrive/MyDrive/models

In [ ]:
!ls drive/MyDrive/models

In [ ]:
!zip -r test.zip /content/model_save/

In [ ]:
files.download("test.zip")